In [68]:
##This file scrapes zillow.com for zpids based on zipcodes and home specs: 4 bd properties. File is saved as propertiesByZip.json

In [69]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import time

In [70]:
#Read CSV file

In [71]:
schoolZipcodes_csv = "../datasets/schoolRanking_medianPrice.csv"
schoolZipcodes_df = pd.read_csv(schoolZipcodes_csv)
schoolZipcodes_df

,Unnamed: 0,Name,Ranking,Address,zipcode,median,latitude,longitude
0,0,Canyon Crest Academy,1,"5951 Village Center Loop Rd.San Diego, CA 92130",92130,896100,32.944802,-117.199324
1,1,Torrey Pines High School,2,"3710 Del Mar Heights Rd.San Diego, CA 92130",92130,896100,32.944802,-117.199324
2,2,Westview High School,3,"13500 Camino Del SurSan Diego, CA 92129",92129,652300,32.966089,-117.127301
3,3,Del Norte High School,4,"16601 Nighthawk Ln.San Diego, CA 92127",92127,683300,33.022206,-117.126479
4,4,Coronado High School,5,"650 D Ave.Coronado, CA 92118",92118,1424100,32.645759,-117.160716
5,5,San Dieguito High Academy,6,"800 Santa Fe Dr.Encinitas, CA 92024",92024,803000,33.050487,-117.251443
6,6,Grossmont Middle College High School,7,"8800 Grossmont College Dr.El Cajon, CA 92020",92020,416600,32.793893,-116.967205
7,7,Sage Creek High School,8,"3900 Cannon Rd.Carlsbad, CA 92010",92010,560600,33.155305,-117.279802
8,8,Rancho Bernardo High School,9,"13010 Paseo LucidoSan Diego, CA 92128",92128,526900,32.995363,-117.073767
9,9,Preuss School UCSD,10,"9500 Gilman Dr.La Jolla, CA 92037",92037,1282800,32.870045,-117.249522


In [72]:
#Create list of unique zipcodes
zipcodes = schoolZipcodes_df['zipcode']
zipcodes.head()

0    92130
1    92130
2    92129
3    92127
4    92118
Name: zipcode, dtype: int64

In [73]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome',**executable_path, headless=False)
propertiesByZip = {}

In [74]:
#Scrape zillow url by looping thru each zipcode in zipcodes
for zipcode in zipcodes:
    zillow_url = 'https://www.zillow.com/'
    zip_url = f'{zillow_url}ca-{zipcode}'
    browser.visit(zip_url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    div = soup.find('div',id='search-results',class_='pop-bubble')
    #print(div)
    ul = div.find('ul',class_='photo-cards')
    #print(ul)
    zpids = []
    li_list = ul.find_all('li')
    for li in li_list:
        article = li.find(attrs = {'data-sgapt': 'For Sale (Broker)'})
        if (article != None):
            span = article.find("span", class_="zsg-photo-card-info")
            if (span != None):
                #print(f'span text: {span.text}')
                if (span.text.startswith( '4 bds' )):
                    zpid = article.attrs['data-zpid']
                    zpids.append(zpid)
    zpidByZip[zipcode] = zpids
    print(zipcode)
    print (zpids)
    #Put in time delay to trick zillow from thinking site is being scraped
    #https://stackoverflow.com/questions/510348/how-can-i-make-a-time-delay-in-python
    time.sleep(30)

92130
['16779400', '48180276', '112747331', '16770847', '51079828', '241607916', '241607233', '16773224']
92130
['16779400', '48180276', '112747331', '16770847', '51079828', '241607916', '241607233', '16773224']
92129
['16807094', '16792457', '16805381', '48180443', '16795198', '16807482', '16792098', '16795758', '16808201']
92127
['69017616', '16740100', '82234010', '54655282', '16740311', '82809154', '59309688', '120287692', '70596957']
92118
['17072977', '17150247', '87792898', '17069962', '87617730', '17151063', '17071546']
92024
['16720405', '16713925', '16716759', '16708860', '16718457', '2087170720']
92020
['17031539', '17018988', '17031778', '17030317', '17018468', '17036368', '17026681', '17054306']
92010
['113501148', '16650197', '16619802', '66110254', '35971781', '241607592', '99514914']
92128
['16737556', '16744164', '16736694', '16798243', '16741941', '16810895', '16797055']
92037
['16848063', '2110940541', '61677715', '87616910']
92129
['16807094', '16792457', '16805381'

In [77]:
#jsonify dictionary
#https://stackoverflow.com/questions/17043860/python-dump-dict-to-json-file
with open('../datasets/zpidByZip.json', 'w') as fp:
    json.dump(propertiesByZip, fp)

In [78]:
print(zpidByZip)

NameError: name 'zpidByZip' is not defined